# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [57]:
#prelims
import polars as pl
import polars.selectors as cs
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data from geodata_prep notebook

In [58]:
#load data from parquet
lf = pl.scan_parquet('port data/dashboard/main.parquet')
#inspect
display(lf.describe())
lf.limit(5).collect()

statistic,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""count""","""1024175""","""1024175""",1.024175e6,1.021926e6,1.013653e6,1.024175e6,"""1024175""",1.024175e6,1.024167e6,1.024175e6,954103.0,952090.0,819386.0,"""1024175""","""1024175""","""1024175""","""1024175""","""1021344""","""1024175""","""1024175""","""1021198""","""1024175""",1.024175e6,1.024175e6,1.024175e6,1.024175e6,1.024175e6,1.024175e6,"""1024175"""
"""null_count""","""0""","""0""",0.0,2249.0,10522.0,0.0,"""0""",0.0,8.0,0.0,70072.0,72085.0,204789.0,"""0""","""0""","""0""","""0""","""2831""","""0""","""0""","""2977""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,"""2021-07-21 14:06:02.223104""",2.986939,184.897959,186.772071,1.768427,null,73.284826,1.0448e7,207.064062,31.613459,10.66267,74.018206,null,null,null,null,"""17:09:03.097916""",null,null,null,null,32.013095,-95.717791,31.974159,-95.897874,31.973375,-95.89908,null
"""std""",null,null,5.500858,96.188133,104.364344,2.394977,null,4.911119,3.1025e7,57.132527,7.462199,2.864933,7.911821,null,null,null,null,null,null,null,null,null,6.544798,20.08293,6.501684,19.909045,6.501389,19.908524,null
"""min""","""205042000""","""2018-01-01 01:14:33""",0.0,0.0,0.0,0.0,null,70.0,0.0,101.0,0.0,-12.8,0.0,"""Albany Port District, NY""","""Albany Port District, NY""","""Guaynabo, PR""","""Not in port waters""","""0:00:01""","""00XE""","""ADM Corpus Christi Grain Eleva…","""Anchorage""","""Albany Port District, NY""",12.37642,-170.62387,17.936081,-166.53444,17.938939,-166.549916,"""205042000_Port of Long Beach, …"
"""25%""",null,"""2019-11-10 00:07:14""",0.0,118.5,93.0,0.0,null,70.0,9.313058e6,179.0,27.0,8.7,70.0,null,null,null,null,"""1:18:01""",null,null,null,null,28.57496,-118.20663,28.645767,-118.21277,28.629389,-118.2095,null
"""50%""",null,"""2021-09-23 22:08:26""",0.1,185.2,183.0,0.0,null,70.0,9.462706e6,189.0,32.0,10.5,71.0,null,null,null,null,"""2:41:00""",null,null,null,null,30.21498,-93.84361,30.35667,-93.941127,30.359621,-93.96069,null
"""75%""",null,"""2023-03-20 11:10:25""",3.5,261.8,273.0,5.0,null,80.0,9.682863e6,229.0,32.0,12.8,80.0,null,null,null,null,"""7:45:41""",null,null,null,null,34.34973,-80.11536,34.201663,-80.122222,34.227365,-80.117801,null
"""max""","""725019340""","""2024-09-30 23:22:34""",102.3,359.9,359.0,15.0,null,89.0,9.901843e8,667.0,86.0,25.5,191.0,"""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""2191 days, 16:27:17""","""1JHK""","""YUSEN TERMINALS BERTHS 212-221""","""Tie Off""","""Wilmington, NC""",61.24418,144.67134,61.24306,-66.086926,61.23778,-66.096678,"""725019340_Corpus Christi, TX_2…"


mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,str,str,str,duration[μs],str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""205042000""",2022-12-07 23:20:37,9.1,358.1,0.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",2h 34m 41s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.36623,-118.04241,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-08 01:55:18,0.3,83.8,328.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",6d 18h 3m 47s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62433,-118.0504,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-14 19:59:05,0.9,232.6,228.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",4h 35m 49s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62758,-118.05079,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-15 00:34:54,0.0,340.4,340.0,5.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1d 8h 25m 12s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21814,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-16 09:00:06,0.2,81.3,339.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1h 2m 19s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21808,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"


## Generate Stats

In [59]:
#define calls dataframe for building stats
calls_df = (
    lf
    #ensure sorting by vessel and time
    .sort(['mmsi', 'time'])
    #grouby by call id
    .group_by('call_id')
    .agg(
        #port name
        port_name = pl.first('port_name'),
        #dock name
        dock_name = pl.first('dock_name'),
        #dock_id
        dock_id = pl.first('dock_id'),
        #vessel mmsi
        mmsi = pl.first('mmsi'),
        #vessel imo
        imo = pl.first('imo'),
        #vessel size
        vessel_size = pl.first('length'),
        #time entering port waters
        time_port_entry = pl.col('time').min(),
        #time of arrival at dock
        time_arrival = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
        ).min(),
        #time of departure from dock
        time_departure = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time')+pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).max(),
        #time port exit
        time_port_exit = pl.col('time').max(),
        #hrs a berth
        hrs_at_berth = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().dt.total_minutes()/60
        ),
        #hrs at anchor
        hrs_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().dt.total_minutes()/60
        )
    )
    #compute additional stats
    .with_columns(
        #time from port entry to docking in hrs
        hrs_to_dock = (
            (pl.col('time_arrival') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        ),
        #time in port waters after leaving dock
        hrs_in_port_after_dock = (
            (pl.col('time_port_exit') - pl.col('time_departure'))
            .dt.total_minutes()/60
        ),
        #total time in port waters in hrs
        hrs_in_port_waters =(
            (pl.col('time_port_exit') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        )
    )
    #collect
    .collect()
)

#inspect
display(calls_df.describe())
calls_df.head()

statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""155558""","""155558""","""155558""","""155558""","""155558""",155556.0,155558.0,"""155558""","""155558""","""155266""","""155558""",155558.0,155558.0,155558.0,155266.0,155558.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",2.0,0.0,"""0""","""0""","""292""","""0""",0.0,0.0,0.0,292.0,0.0
"""mean""",null,null,null,null,null,1.0139e7,209.068225,"""2021-06-08 09:08:02.359814""","""2021-06-09 13:46:17.326894""","""2021-06-13 06:00:57.885036""","""2021-06-15 06:48:52.969593""",69.889625,14.35373,28.629654,33.688093,165.672578
"""std""",null,null,null,null,null,2.6462e7,59.507905,null,null,null,null,571.122601,169.387675,641.291639,1379.80125,1857.174937
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 01:14:33""","""2018-01-01 01:43:12""","""2018-01-01 04:23:54""","""2018-01-01 09:13:38""",0.0,0.0,0.0,-36093.2,0.0
"""25%""",null,null,null,null,null,9.298507e6,177.0,"""2019-08-28 23:09:30""","""2019-08-30 04:02:05""","""2019-09-01 23:05:51""","""2019-09-05 13:47:11""",16.683333,0.0,2.683333,0.0,22.65
"""50%""",null,null,null,null,null,9.402316e6,191.0,"""2021-07-24 20:39:21""","""2021-07-26 14:02:25""","""2021-07-31 17:18:51""","""2021-08-02 10:31:29""",31.566667,0.0,3.633333,0.0,40.75
"""75%""",null,null,null,null,null,9.61385e6,233.0,"""2023-02-27 10:39:07""","""2023-02-28 10:58:32""","""2023-03-04 03:39:06""","""2023-03-07 00:24:39""",59.083333,0.0,5.866667,0.816667,79.05
"""max""","""725019340_Corpus Christi, TX_2…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019340""",9.901843e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:48:33""","""2024-09-30 23:22:34""","""2024-09-30 23:22:34""",58940.316667,52600.45,58343.5,58107.066667,59115.216667


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""477195600_Mobile, AL_2021-03-0…","""Mobile, AL""","""ALABAMA STATE DOCKS DEPARTMENT…","""0SHB""","""477195600""",9.580039e6,175.0,2021-03-05 02:38:36,2021-03-05 09:33:00,2021-03-05 20:49:24,2021-03-05 20:49:24,11.266667,0.0,6.9,0.0,18.166667
"""255806068_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 33A, 33…","""0YT0""","""255806068""",9.395111e6,166.0,2019-01-29 22:24:06,2019-01-30 00:40:50,2019-01-30 13:12:14,2019-01-30 14:59:58,12.516667,0.0,2.266667,1.783333,16.583333
"""311054300_Port of Brunswick, G…","""Port of Brunswick, GA""","""GEORGIA PORTS AUTHORITY, COLON…","""0S0Q""","""311054300""",9.441594e6,199.0,2019-05-23 08:45:54,2019-05-23 12:23:14,2019-05-24 04:22:41,2019-05-24 04:22:41,15.533333,0.0,3.616667,0.0,19.6
"""357240000_Port of Long Beach, …","""Port of Long Beach, CA""","""HANJIN SHIPPING CO. (LONG BEAC…","""1059""","""357240000""",9.404675e6,365.0,2021-04-09 20:14:39,2021-04-14 13:56:19,2021-04-22 01:54:43,2021-04-22 01:54:43,178.316667,107.283333,113.683333,0.0,293.666667
"""255806063_Port Everglades, FL_…","""Port Everglades, FL""","""PORT EVERGLADES BERTHS 30, 31 …","""0YSY""","""255806063""",9.43582e6,138.0,2022-02-02 12:37:01,2022-02-02 23:29:47,2022-02-03 00:29:53,2022-02-03 02:28:31,1.0,0.0,10.866667,1.966667,13.85


## NOTES on calls_df

- hrs_in_port_after_dock clearly not working as expected; unsure why
    - not using this stat at the moment so we can fix later if needed
- 0 values for hrs_in_port_waters, hrs_at_berth, etc unexpected 
    - likely edge cases as percentiles seem reasonable
     


In [54]:
#drop calls with unrealistic times
calls_df = (
    calls_df.filter(
        #zero time in port waters
        pl.col('hrs_in_port_waters') == 0,
        #zero time at berth
        pl.col('hrs_at_berth') == 0,
        #zero time to dock
        #pl.col('hrs_to_dock') == 0,
    )
)

In [55]:
#create monthly stats dataframe
monththly_df = (
    calls_df
    #get month from docking time
    .with_columns(
        #extract month from docking time
        month = pl.col('time_arrival').dt.strftime('%Y%m')
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_occupied = pl.sum('hrs_at_berth'),
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor'),
        #time in port waters 
        hrs_in_port_waters_total = pl.sum('hrs_in_port_waters'),
        hrs_in_port_waters_mean = pl.mean('hrs_in_port_waters'),
        hrs_in_port_waters_median = pl.median('hrs_in_port_waters')
    )
    #get hours from each month
    .with_columns(
        hrs_in_month = (
            pl.when(pl.col('month').str.tail(2).is_in(['01', '03', '05', '07',
                                                       '08', '10', '12']))
            .then(31*24)
            .when(pl.col('month').str.tail(2).is_in(['04', '06', '09', '11']))
            .then(30*24)
            .otherwise(28*24)
        )
    )
    .with_columns(
        #dock utilization - percentage of time a dock is occupied
        utilization = (
            pl.col('hrs_occupied')/pl.col('hrs_in_month')
        )
    )
    #drop hours in month
    .drop('hrs_in_month')
    #sort by port dock then month
    .sort(['port_name', 'dock_id', 'month'])
)

#inspect
display(monththly_df.describe())
monththly_df.head()

statistic,port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""64""","""64""","""64""",64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0,64.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,1.03125,209.484375,1.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""std""",null,null,null,0.175368,57.17947,0.175368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""min""","""Anacortes, WA""","""086Z""","""202007""",1.0,110.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,null,1.0,175.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""50%""",null,null,null,1.0,199.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""75%""",null,null,null,1.0,238.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""max""","""Wilmington, NC""","""11ZS""","""202311""",2.0,369.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,u32,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Anacortes, WA""","""0U5U""","""202304""",1,159.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Baltimore, MD""","""0XHG""","""202008""",1,211.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Baltimore, MD""","""0Y1W""","""202009""",1,199.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Baltimore, MD""","""0Y35""","""202007""",1,228.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Beaumont, TX""","""11ZP""","""202304""",1,182.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## NOTES on monthly stats
- As in calls_df, 0 values for hrs stats unexpected
    - docks that do not see any vessels do not appear in the underlying data since that's based on AIS moored statuses being matched within 500m of the dock
    - percentiles look reasonable so these may be edge cases with little effect on the stats
    - possibly drop call_ids with impossible 0 values before generating this dataframe?

#### Hours calc discussion

The current code first associates the call_id with the month in which the vessel arrived at dock, then counts total times for that call_id to that month. This results in some edge cases where hours stats far exceed the total hours in the month, as in the case that a vessel arrives at the dock and stays there for a very long period of time. 

This can be partially resolved by dropping statuses that are very long, which needs to be done anyway.
- what's the right strategy? set status duration to (the median for that dock? zero? 12hr?) and give an unknown status afterwards? 

It would be fully resolved by totaling monthly hrs (at dock or hrs utilized, for example) independently of call_id.  

In [ ]:
monththly_df.with_columns(pl.all().replace(0,None)).describe()

In [ ]:
#create annual port stats dataframe
annual_df = (
    calls_lf
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('docking_time').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('docking_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth')/60,
        hrs_at_berth_mean = pl.mean('hrs_at_berth')/60,
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor')/60,
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')/60
    )
    #sort by port then month
    .sort(['port_name', 'year'])
    #collect
    .collect()
)

#inspect
annual_df.head()

In [ ]:
#get point in time stats

#create point in time (pit) df to join stats to
pit_df = (
    lf
    .with_columns(date = pl.col('time').dt.date())
    .select('port_name', 'dock_id', 'month', 'date')
    .unique().collect()
)

for hour in range(0, 24):
    #create a time object for each hour
    hour_dt = pl.time(hour)
    #create a dataframe for each hour
    hour_df = (
        lf
        .with_columns(
            #get end of status time
            end_time = (pl.col('time') + 
                        pl.duration(minutes=pl.col('status_duration'))),
            #get date from time
            date = pl.col('time').dt.date()
        )
        #group by port dock and hour
        .group_by(['port_name', 'dock_id', 'month', 'date'])
        .agg(
            #number of vessels at dock at each hour
            vessels_at_dock = (
                #when moored at hour
                pl.when((pl.col('status')==5) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                         pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls() #n_unique counts nulls as unique values
                .n_unique()
            ),
            #number of vessels at anchor at each hour
            vessels_at_anchor = (
                #when anchored at hour
                pl.when((pl.col('status')==1) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                        pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls()
                .n_unique()
            )
        )
        .collect()
    )
    #join the hour dataframe to the main dataframe
    pit_df = (
        pit_df
        .join(hour_df, 
              on=['port_name', 'dock_id', 'month', 'date'], 
              how='left')
        #rename the columns to include the hour
        .rename({
            'vessels_at_dock': f'vessels_at_dock_{hour}',
            'vessels_at_anchor': f'vessels_at_anchor_{hour}'
        })
    )

#get port stats by month
pit_df = (
    pit_df
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #sum the number of vessels at dock at each hour
        cs.starts_with('vessels_at_dock_').sum(),
        #sum the number of vessels at anchor at each hour
        cs.starts_with('vessels_at_anchor_').sum()
    )
    #get the max at any hour
    .with_columns(
        #get max at dock at any hour
        vessels_at_dock_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get mean at dock any hour
        vessels_at_dock_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get max at anchor at any hour
        vessels_at_anchor_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_anchor_'))
        ),
        #get mean at anchor any hour
        vessels_at_anchor_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_anchor_'))
        )
    )
    #select the columns to keep
    .select(['port_name', 'month', 'date', 'vessels_at_dock_max', 
             'vessels_at_dock_mean', 'vessels_at_anchor_max',
             'vessels_at_anchor_mean'])
    #aggregate by month
    .group_by(['port_name', 'month'])
    .agg(
        #get max at anchor on any date
        vessels_at_anchor_max = pl.max('vessels_at_anchor_max'),
        #get mean at anchor on any date
        vessels_at_anchor_mean = pl.mean('vessels_at_anchor_mean'),
        #get max at dock on any date
        vessels_at_dock_max = pl.max('vessels_at_dock_max'),
        #get mean at dock on any date
        vessels_at_dock_mean = pl.mean('vessels_at_dock_mean'
        )
    )
)

In [ ]:
#inspect
pit_df.head()

In [ ]:



    
    #group by month to get averages
    .group_by(['port_name', 'month'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor')
    )
    #sort by port then date
    .sort(['port_name', 'month'])
    #collect
    .collect()
)

#inspect
df.head()

#join to stats dfs



In [ ]:
#get means for each port over all time
port_alltime_df = (
    port_monththly_df
    .group_by(['port_name'])
    .agg(
        #average number of vessels at dock
        vessels_at_dock_at_noon_mean = pl.mean('vessels_at_dock_at_noon_mean'),
        #average number of vessels at anchor
        vessels_at_anchor_at_noon_mean = pl.mean('vessels_at_anchor_at_noon_mean'),
        #average time at dock in hours
        hrs_at_berth_median = pl.mean('hrs_at_berth_median'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth_mean'),
        #average time at anchor in hours
        hrs_at_anchor_median = pl.mean('hrs_at_anchor_median'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor_mean')
    )
    #sort by port
    .sort(['port_name'])
)

#inspect
port_alltime_df.head()